# Demo: Frequency Response (Part 1)

#### Import modules and configure the notebook.

In [ ]:
# These are standard modules
import time
import numpy as np
import sympy as sym
from scipy.signal import place_poles
from scipy import linalg
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from IPython.display import display, Latex, Markdown

# This is a custom interface to the pybullet simulator
import ae353_wheel_sensors

# Stupid hack because tornado is broken in jupyter
%matplotlib widget
plt.ioff()
plt.ion()

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

#### Choose parameter values

The equation of motion is

$$c_1 \ddot{q} = \tau - (c_2 \sin \phi) \sin q$$

where

* $q$ is the wheel angle
* $\tau$ is the wheel torque, i.e., the torque applied to the wheel by a motor on the ground
* $\phi$ is the "roll angle" of the ground with respect to horizontal

and where the values of $c_1$ and $c_2$ are given as follows:

In [ ]:
# Values of c1 and c2 for the example (do not change)
c1 = 1.0
c2 = 2.0

# Values of other parameters that are given
g = 9.81 # <-- acceleration of gravity
r = 0.35 # <-- distance from wheel axis to marker (i.e., to the mass)

# Values of other parameters that correspond to the coefficients c1 and c2 that we chose
m = c2 / (g * r)   # <-- set in code
J = c1 - m * r**2  # <-- set in URDF
print(f'm = {m}\nJ = {J}')

#### Set up simulations

Create an instance of the `Simulator` class with particular values of `roll`, `mass`, and `inertia`.

In [ ]:
simulator = ae353_wheel_sensors.Simulator(
    display=True,
    roll=0.,
    mass=m,
    inertia=J,
)

#### Define functions to plot results

This function plots results in terms of nonlinear states and inputs.

In [ ]:
def show_results(data, q_e, v_e, tau_e):
    t = data['t']
    q = data['wheel_angle']
    v = data['wheel_velocity']
    tau = data['wheel_torque']
    tau_cmd = data['wheel_torque_command']
    q_meas = data['wheel_angle_measurement']
    q_des = data['desired_wheel_angle']
    xhat = data['xhat']
    q_est = xhat[:, 0] + q_e
    v_est = xhat[:, 1] + v_e

    fig, (ax_q, ax_v, ax_tau) = plt.subplots(
        3, 1, figsize=(10, 8), sharex=True,
    )

    ax_q.plot(t, q, label=r'$q$', linewidth=4)
    ax_q.plot(t, q_est, '--', label=r'$q$ (estimated)', linewidth=4)
    ax_q.plot(t, q_meas, '.', markersize=1, label=r'$q$ (measured)')
    ax_q.plot(t, q_des, ':', label=r'$q$ (desired)')
    ax_q.plot(t, q_e * np.ones_like(t), ':', label=r'$q_e$', linewidth=3, color='C3')
    ax_v.plot(t, v, label=r'$v$', linewidth=4)
    ax_v.plot(t, v_est, '--', label=r'$v$ (estimated)', linewidth=4)
    ax_v.plot(t, v_e * np.ones_like(t), ':', label=r'$v_e$', linewidth=3, color='C3')
    ax_tau.plot(t, tau, label=r'$\tau$', linewidth=4)
    ax_tau.plot(t, tau_cmd, '-.', label=r'$\tau$ (command)', linewidth=3, color='C6')
    tau_max = np.ones_like(t) * simulator.tau_max
    ax_tau.plot(t, tau_max,
                '--', label=r'$\tau$ (max)', linewidth=2, color='C2', zorder=0)
    ax_tau.plot(t, - tau_max,
                '--', linewidth=2, color='C2', zorder=0)

    ax_q.grid()
    ax_q.legend(fontsize=14, ncol=4, loc='upper right')
    ax_q.tick_params(labelsize=14)
    ax_q.set_ylim(np.min(q_des) - 0.5, np.max(q_des) + 0.5)

    ax_v.grid()
    ax_v.legend(fontsize=14, ncol=3, loc='upper right')
    ax_v.tick_params(labelsize=14)
    ax_v.set_ylim(v_e - 1., v_e + 1.)

    ax_tau.grid()
    ax_tau.legend(fontsize=14, ncol=3, loc='upper right')
    ax_tau.tick_params(labelsize=14)
    ax_tau.set_ylim(-1.2 * simulator.tau_max, 1.2 * simulator.tau_max)

    ax_tau.set_xlabel('time (s)', fontsize=14)
    ax_tau.set_xlim([data['t'][0], data['t'][-1]])

    fig.tight_layout()
    plt.show()

    plt.ioff()
    plt.ion()

This function plots only $q$ and $q_\text{des}$.

In [ ]:
def show_results_q(data, q_e, v_e, tau_e, omega=None, mag=None, ang=None, linewidth=4):
    t = data['t']
    q = data['wheel_angle']
    q_des = data['desired_wheel_angle']
    q_est = data['xhat'][:, 0] + q_e
    if mag is not None:
        q_pre = mag * np.sin(omega * data['t'] + ang)
    fig, ax_q = plt.subplots(
        1, 1, figsize=(8, 4), sharex=True,
    )

    ax_q.plot(t, q, label=r'$q$', linewidth=linewidth, color='C0')
    ax_q.plot(t, q_des, '--', label=r'$q$ (desired)', linewidth=linewidth, color='C1')
    ax_q.plot(t, q_est, '-.', label=r'$q$ (estimated)', linewidth=linewidth-2, color='C2')
    if mag is not None:
        ax_q.plot(t, q_pre, ':', label=r'$q$ (predicted)', linewidth=linewidth-1, color='C6')
    ax_q.grid()
    ax_q.legend(fontsize=14, ncol=4, loc='upper right')
    ax_q.tick_params(labelsize=14)
    ax_q.set_ylim(-0.2, 0.2)
    ax_q.set_xlabel('time (s)', fontsize=14)
    ax_q.set_xlim([data['t'][0], data['t'][-1]])

    fig.tight_layout()
    plt.show()

    plt.ioff()
    plt.ion()

#### Define functions to print numerical results nicely

In [ ]:
def show_matrix(key, val):
    latex = sym.latex(sym.nsimplify(sym.Matrix(val), rational=True))
    display(Markdown(f'$${key} = {latex}$$'))
    
def show_transfer_function(key, val):
    latex = sym.latex(sym.nsimplify(val, rational=True))
    display(Markdown(f'$${key}(s) = {latex}$$'))

#### Do controller design

Derive state-space model.

In [ ]:
# Define symbolic variables
q, v, tau = sym.symbols('q, v, tau')

# Define ODEs
f = sym.Matrix([
    v,
    tau / c1,
])

# Convert floats to rationals
f = sym.nsimplify(f, rational=True)

# Choose equilibrium point
(q_e, v_e, tau_e) = (0., 0., 0.)

# Linearize
A_num = sym.lambdify([q, v, tau], f.jacobian([q, v]))
A = A_num(q_e, v_e, tau_e).astype(float)
B_num = sym.lambdify([q, v, tau], f.jacobian([tau]))
B = B_num(q_e, v_e, tau_e).astype(float)

# Show result
show_matrix('A', A)
show_matrix('B', B)

Design controller by eigenvalue placement.

In [ ]:
K = signal.place_poles(A, B, [-1. + 4. * 1j, -1. - 4. * 1j]).gain_matrix
show_matrix('K', K)

#### Do observer design

Derive state-space model.

In [ ]:
# Define nonlinear model
g = sym.Matrix([q])

# Convert floats to rationals
g = sym.nsimplify(g, rational=True)

# Linearize
C_num = sym.lambdify([q, v, tau], g.jacobian([q, v]))
C = C_num(q_e, v_e, tau_e).astype(float)
D_num = sym.lambdify([q, v, tau], g.jacobian([tau]))
D = D_num(q_e, v_e, tau_e).astype(float)

# Show result
show_matrix('C', C)
show_matrix('D', D)

Design observer by eigenvalue placement.

In [ ]:
L = signal.place_poles(A.T, C.T, [-6., -7.]).gain_matrix.T
show_matrix('L', L)

#### Implement and test

Implement.

In [ ]:
class Controller:
    def __init__(self, A, B, C, K, L, q_e, v_e, tau_e, get_desired_wheel_angle):
        self.A = A
        self.B = B
        self.C = C
        self.K = K
        self.L = L
        self.q_e = q_e
        self.v_e = v_e
        self.tau_e = tau_e
        
        self.dt = 0.01

        self.get_desired_wheel_angle = get_desired_wheel_angle
        
        self.variables_to_log = ['xhat', 'desired_wheel_angle']
    
    def reset(self):
        self.desired_wheel_angle = self.get_desired_wheel_angle(0.)
        self.xhat = np.array([
            self.desired_wheel_angle - self.q_e,
            0.,
        ])
    
    def run(self, t, wheel_angle_measurement):
        """
        INPUTS
         t = current time (s)
         wheel_angle_measurement = measured angle of wheel from sensors (rad)
         
        OUTPUTS
         wheel_torque = torque applied to wheel by motor on ground (N m)
        """
        
        # Get desired wheel angle
        self.desired_wheel_angle = self.get_desired_wheel_angle(t)
        
        # Get desired state
        xdes = np.array([
            self.desired_wheel_angle - self.q_e,
            self.v_e                 - self.v_e,
        ])
        
        # Get input
        u = - self.K @ (self.xhat - xdes)
        
        # Get measured wheel angle (sensors)
        q = wheel_angle_measurement
        
        # Get output
        y = np.array([
            q - self.q_e,
        ])
        
        # Get state estimate
        self.xhat += self.dt * (
            self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y)
        )
        
        # Get torque (actuators)
        wheel_torque = u[0] + self.tau_e
        
        return wheel_torque

Test controller and observer (with tracking), given a desired wheel angle of the form

$$q_\text{des}(t) = a$$

for some constant $a$.

In [ ]:
# Choose amplitude
a = 0.

# Define function to get desired wheel angle
get_desired_wheel_angle = lambda t: a

# Create controller
controller = Controller(A, B, C, K, L, q_e, v_e, tau_e, get_desired_wheel_angle)

# Choose initial conditions
(q_i, v_i) = (get_desired_wheel_angle(0.) + 0.15, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,
)

# Reset controller
controller.reset()

# Run simulator
simulator.display_meshcat = True
data = simulator.run(
    controller,
    max_time=5.,
)

# Show results
# show_results(data, q_e, v_e, tau_e)
show_results_q(data, q_e, v_e, tau_e)

Test controller and observer (with tracking), given a desired wheel angle of the form

$$q_\text{des}(t) = a \sin(\omega t)$$

where $a$ is chosen small enough so that our state-space model is still accurate. Remember that the period $T$ (in seconds) of a sinusoid with frequency $\omega$ (in radians / second) is

$$T = \dfrac{2\pi}{\omega}.$$

In [ ]:
# Choose amplitude of oscillation
a = 0.1

# Choose period of oscillation
T = 1.
omega = 2 * np.pi / T

# # Choose frequency of oscillation
# omega = 1.
# T = 2 * np.pi / omega

# Show period and frequency of oscillation
print(f'omega = {omega:.3f} (T = {T:.3f})')

# Define function to get desired wheel angle
get_desired_wheel_angle = lambda t: a * np.sin(omega * t)

# Create controller
controller = Controller(A, B, C, K, L, q_e, v_e, tau_e, get_desired_wheel_angle)

# Choose initial conditions
(q_i, v_i) = (get_desired_wheel_angle(0.) + 0.15, v_e)

# Reset controller
controller.reset()

# Reset simulator
simulator.reset(
    wheel_angle=q_i,
    wheel_velocity=v_i,
    sensor_noise=0.,
)

# Reset controller
controller.reset()

# Run simulator
simulator.display_meshcat = True
data = simulator.run(
    controller,
    max_time=10 * T,
)

# Show results
# show_results(data, q_e, v_e, tau_e)
show_results_q(data, q_e, v_e, tau_e)